## Imports

In [1]:
# IMPORTS
#ML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pycountry
import rasterio
from scipy.spatial import cKDTree
from tqdm import tqdm
from datetime import datetime
import geopandas as gpd

import zipfile
import os
import gdown

# Contry data

In [2]:
country_info_path = "https://drive.google.com/uc?id=1xfYlruvfAi6yieOd_S69pPYWphckRLr5&export=download"

column_names = [
    'Country_Code',        # Alpha-2 code
    'ISO_Alpha_3',        # Alpha-3 code
    'Numeric_Code',        # Numeric code
    'Alpha_2',            # Alpha-2 code (duplicate)
    'Country_Name',       # Name of the country
    'Capital',            # Capital city
    'Area',               # Area in square kilometers
    'Population',         # Population
    'Region',             # Region
    'TLD',                # Top-level domain
    'Currency_Code',      # Currency code
    'Currency_Name',      # Currency name
    'Currency_Numeric',   # Numeric currency code
    'Additional_Info'     # Additional information
]

country_info_df = pd.read_csv(
    country_info_path, delimiter="\t", comment="#", on_bad_lines="skip", header=None
)

country_info_df.columns = column_names
country_info_df = country_info_df[["Country_Code","ISO_Alpha_3"]]
country_info_df.head()

,Country_Code,ISO_Alpha_3
0,AD,AND
1,AE,ARE
2,AI,AIA
3,AL,ALB
4,AM,ARM


## Countries 

In [3]:
countries_zip_url = "https://drive.google.com/uc?id=1UQzdO7suT0BnwKBeNybMG97vM9GIDogA"
countries_zip_file_path = "../../allCountries.zip"

# Download the ZIP file if it doesn't exist; otherwise, proceed to read the TXT file.
if not os.path.exists(countries_zip_file_path):
    gdown.download(countries_zip_url, countries_zip_file_path, quiet=False)

with zipfile.ZipFile(countries_zip_file_path) as z:
    countries_txt_filename = "allCountries.txt"

    with z.open(countries_txt_filename) as txt_file:
        countries_df = pd.read_csv(txt_file, sep="\t", header=None)


# https://download.geonames.org/export/dump/
countries_df.columns = [
    'geonameid',         
    'name',             
    'asciiname',        
    'alternatenames',  
    'latitude',         
    'longitude',       
    'feature class',    
    'feature code',      
    'iso alpha 2',      
    'cc2',              
    'admin1 code',     
    'admin2 code',       
    'admin3 code',      
    'admin4 code',   
    'population',      
    'elevation',       
    'dem',             
    'timezone',          
    'modification date'  
]
countries_df = countries_df[[
    'geonameid',         
    'name',             
    'asciiname',        
    'latitude',         
    'longitude',       
    'iso alpha 2'
]]

print(f"\nshape: {countries_df.shape}")
countries_df.head()

/var/folders/9m/ym86jvl93wq3tx6ssy8ql7kh0000gn/T/ipykernel_51482/1737397109.py:12: DtypeWarning: Columns (9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  countries_df = pd.read_csv(txt_file, sep="\t", header=None)



shape: (12950185, 6)


,geonameid,name,asciiname,latitude,longitude,iso alpha 2
0,2994701,Roc Meler,Roc Meler,42.58765,1.74180,AD
1,3017832,Pic de les Abelletes,Pic de les Abelletes,42.52535,1.73343,AD
2,3017833,Estany de les Abelletes,Estany de les Abelletes,42.52915,1.73362,AD
3,3023203,Port Vieux de la Coume d’Ose,Port Vieux de la Coume d'Ose,42.62568,1.61823,AD
4,3029315,Port de la Cabanette,Port de la Cabanette,42.60000,1.73333,AD


## EUI

In [4]:
eui_url = "https://drive.google.com/uc?id=12qGq_DLefI1RihIF_RKQUyJtm480-xRC"
eui_df = pd.read_csv(eui_url)
print(f"shape: {eui_df.shape}")

merged_df = pd.merge(
    countries_df, eui_df, left_on="geonameid", right_on="Geonames ID", how="inner"
)
assert merged_df.shape[0] == eui_df.shape[0]
print(f"shape: {merged_df.shape}")
merged_df.head()

shape: (482, 5)
shape: (482, 11)


,geonameid,name,asciiname,latitude,longitude,iso alpha 2,City,Geonames ID,Country,Residential EUI (kWh/m2/year),Non-residential EUI (kWh/m2/year)
0,292968,Abu Dhabi,Abu Dhabi,24.45118,54.39696,AE,Abu Dhabi,292968,United Arab Emirates,128.447899,226.725457
1,1138958,Kabul,Kabul,34.52813,69.17233,AF,Kabul,1138958,Afghanistan,213.167026,144.395840
2,3183875,Tirana,Tirana,41.32750,19.81889,AL,Tirana,3183875,Albania,133.717672,101.873579
3,616052,Yerevan,Yerevan,40.18111,44.51361,AM,Yerevan,616052,Armenia,198.865302,132.124738
4,2240449,Luanda,Luanda,-8.83682,13.23432,AO,Luanda,2240449,Angola,65.343750,104.311100


In [5]:
usa_eui_grouped_path = '../data/02_interim/eui_usa_cities_grouped_df.csv'
usa_eui_grouped_df = pd.read_csv(usa_eui_grouped_path)
print(f"shape: {usa_eui_grouped_df.shape}")

merged_usa_df = pd.merge(
    countries_df, usa_eui_grouped_df, left_on="geonameid", right_on="Geonames ID", how="inner"
)

print(f"shape: {merged_usa_df.shape}")
merged_usa_df.head()

shape: (8, 5)
shape: (8, 11)


,geonameid,name,asciiname,latitude,longitude,iso alpha 2,City,Geonames ID,Country,Residential EUI (kWh/m2/year),Non-residential EUI (kWh/m2/year)
0,4164138,Miami,Miami,25.77427,-80.19366,US,Miami,4164138,United States of America,131.335868,207.250764
1,4560349,Philadelphia,Philadelphia,39.95238,-75.16362,US,Philadelphia,4560349,United States of America,192.884682,305.366553
2,4887398,Chicago,Chicago,41.85003,-87.65005,US,Chicago,4887398,United States of America,262.185687,327.467342
3,4930956,Boston,Boston,42.35843,-71.05977,US,Boston,4930956,United States of America,197.244155,350.017793
4,5059296,Fullerton,Fullerton,46.16525,-98.42704,US,Fullerton,5059296,United States of America,101.008903,196.081083


In [6]:
merged_df["source"] = "World Bank CURB"
merged_usa_df["source"] = "City Energy Benchmark"

merged_df = pd.concat([merged_df, merged_usa_df], ignore_index=True)

print(f"shape: {merged_df.shape}")
merged_df.head()

shape: (490, 12)


,geonameid,name,asciiname,latitude,longitude,iso alpha 2,City,Geonames ID,Country,Residential EUI (kWh/m2/year),Non-residential EUI (kWh/m2/year),source
0,292968,Abu Dhabi,Abu Dhabi,24.45118,54.39696,AE,Abu Dhabi,292968,United Arab Emirates,128.447899,226.725457,World Bank CURB
1,1138958,Kabul,Kabul,34.52813,69.17233,AF,Kabul,1138958,Afghanistan,213.167026,144.395840,World Bank CURB
2,3183875,Tirana,Tirana,41.32750,19.81889,AL,Tirana,3183875,Albania,133.717672,101.873579,World Bank CURB
3,616052,Yerevan,Yerevan,40.18111,44.51361,AM,Yerevan,616052,Armenia,198.865302,132.124738,World Bank CURB
4,2240449,Luanda,Luanda,-8.83682,13.23432,AO,Luanda,2240449,Angola,65.343750,104.311100,World Bank CURB


## Adding ISO 3 Code

In [7]:
# Adding ISO CODE 3
alpha_2_to_alpha_3 = {country.alpha_2: country.alpha_3 for country in pycountry.countries}
merged_df.loc[:, 'ISO_alpha3'] = merged_df['iso alpha 2'].map(alpha_2_to_alpha_3)

# Manually correcting the missing country code for Namibia by assigning 'NAM' because country code is null for Nambia
merged_df.loc[merged_df['Country'] == 'Namibia', 'ISO_alpha3'] = 'NAM'
merged_df.loc[merged_df['Country'] == 'Namibia', 'iso alpha 2'] = 'NA'

print(f"shape: {merged_df.shape}")

assert merged_df["ISO_alpha3"].isna().sum() == 0, "There are missing values in the ISO_alpha3 column."
assert merged_df.shape[0] == 490, "The number of rows in merged_df is not 482 + 8."

shape: (490, 13)


# Population (Country Level)

In [8]:
population_path = '../data/01_raw/population.csv'
population_df = pd.read_csv(population_path, skiprows=4)
population_2023 = population_df[['Country Code', '2023']]

population_2023.rename(columns={
    '2023': 'Population_2023',
    'Country Code': 'ISO_alpha3'
}, inplace=True)

#taiwan
taiwan_raw = {'Country Name': 'Taiwan', 'ISO_alpha3': 'TWN', 'Population_2023': 23894394}
population_2023.loc[len(population_2023)] = taiwan_raw

merged_df = merged_df.merge(population_2023, on='ISO_alpha3', how='left')

assert merged_df['Population_2023'].notnull().all(), "Error: There are null values in 'Population_2023'."
assert merged_df.shape[0] == 490, "The number of rows in merged_df is not 482 + 8."
print(f"shape: {merged_df.shape}")
merged_df.head()

shape: (490, 14)


/var/folders/9m/ym86jvl93wq3tx6ssy8ql7kh0000gn/T/ipykernel_51482/3187838866.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population_2023.rename(columns={
/var/folders/9m/ym86jvl93wq3tx6ssy8ql7kh0000gn/T/ipykernel_51482/3187838866.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population_2023.loc[len(population_2023)] = taiwan_raw


,geonameid,name,asciiname,latitude,longitude,iso alpha 2,City,Geonames ID,Country,Residential EUI (kWh/m2/year),Non-residential EUI (kWh/m2/year),source,ISO_alpha3,Population_2023
0,292968,Abu Dhabi,Abu Dhabi,24.45118,54.39696,AE,Abu Dhabi,292968,United Arab Emirates,128.447899,226.725457,World Bank CURB,ARE,9516871.0
1,1138958,Kabul,Kabul,34.52813,69.17233,AF,Kabul,1138958,Afghanistan,213.167026,144.395840,World Bank CURB,AFG,42239854.0
2,3183875,Tirana,Tirana,41.32750,19.81889,AL,Tirana,3183875,Albania,133.717672,101.873579,World Bank CURB,ALB,2745972.0
3,616052,Yerevan,Yerevan,40.18111,44.51361,AM,Yerevan,616052,Armenia,198.865302,132.124738,World Bank CURB,ARM,2777970.0
4,2240449,Luanda,Luanda,-8.83682,13.23432,AO,Luanda,2240449,Angola,65.343750,104.311100,World Bank CURB,AGO,36684202.0


# Population Density

In [9]:
pop_density_path = '../data/02_interim/population_density.csv'
pop_density_df = pd.read_csv(pop_density_path)
pop_density_df = pop_density_df[['geonameid','population_density']]
pop_density_df = pop_density_df.drop_duplicates() # seattle and other cities we have duplicates
merged_df = merged_df.merge(pop_density_df, on='geonameid', how='left')
assert merged_df['population_density'].notnull().all()
print(f"shape: {merged_df.shape}")
merged_df.head()

shape: (490, 15)


,geonameid,name,asciiname,latitude,longitude,iso alpha 2,City,Geonames ID,Country,Residential EUI (kWh/m2/year),Non-residential EUI (kWh/m2/year),source,ISO_alpha3,Population_2023,population_density
0,292968,Abu Dhabi,Abu Dhabi,24.45118,54.39696,AE,Abu Dhabi,292968,United Arab Emirates,128.447899,226.725457,World Bank CURB,ARE,9516871.0,6027.494801
1,1138958,Kabul,Kabul,34.52813,69.17233,AF,Kabul,1138958,Afghanistan,213.167026,144.395840,World Bank CURB,AFG,42239854.0,40085.123056
2,3183875,Tirana,Tirana,41.32750,19.81889,AL,Tirana,3183875,Albania,133.717672,101.873579,World Bank CURB,ALB,2745972.0,26918.855498
3,616052,Yerevan,Yerevan,40.18111,44.51361,AM,Yerevan,616052,Armenia,198.865302,132.124738,World Bank CURB,ARM,2777970.0,13532.447017
4,2240449,Luanda,Luanda,-8.83682,13.23432,AO,Luanda,2240449,Angola,65.343750,104.311100,World Bank CURB,AGO,36684202.0,37139.850778


# HDI - Educational Index - Income Index.csv

In [10]:
HDI_EI_II_path = '../data/01_raw/HDI_educationalIndex_incomeIndex.csv'
HDI_EI_II_df = pd.read_csv(HDI_EI_II_path)
HDI_EI_II_df = HDI_EI_II_df[["ISO_Code", "Subnational HDI","Educational index", "Income index"]]
HDI_EI_II_df.rename(columns={'ISO_Code':'ISO_alpha3' }, inplace=True)


merged_df = merged_df.merge(HDI_EI_II_df, 
                             on='ISO_alpha3', 
                             how='left')

assert merged_df.loc[merged_df['Country'] != 'Taiwan', 
                     ['Subnational HDI', 'Educational index', 'Income index']].notnull().all().all()
assert merged_df.shape[0] == 490, "The number of rows in merged_df is not 482 + 8."
print(f"shape: {merged_df.shape}")

shape: (490, 18)


# GDP

In [11]:
gdp_data_path = (
    "../data/01_raw/gdp_data.csv"
)
gdp_df = pd.read_csv(gdp_data_path)
gdp_df = gdp_df[gdp_df["Level"]=="National"]

gdp_df = gdp_df[['ISO_Code', '2022']]
gdp_df.rename(columns={'2022': 'GDP_2022', 'ISO_Code':'ISO_alpha3' }, inplace=True)


merged_df = merged_df.merge(gdp_df, 
                             on='ISO_alpha3', 
                             how='left')

assert merged_df.loc[merged_df['Country'] != 'Taiwan', 
                     ['GDP_2022']].notnull().all().all()
assert merged_df.shape[0] == 490, "The number of rows in merged_df is not 482 + 8."
print(f"shape: {merged_df.shape}")

shape: (490, 19)


# Urbanization rate

In [12]:
# Load the Urbanization Rate dataset, skipping metadata rows if necessary
urbanization_rate_path = (
    "https://drive.google.com/uc?id=1YteyPHAWnJUKG0LWogS98EYnwjRTeZDf&export=download"
)
urbanization_rate_df = pd.read_csv(urbanization_rate_path, skiprows=4)


urbanization_rate_df = urbanization_rate_df[["Country Code", "2022"]].rename(
    columns={"2022": "Urbanization_Rate_2022" , 'Country Code':'ISO_alpha3'}
)

merged_df = merged_df.merge(urbanization_rate_df, 
                             on='ISO_alpha3', 
                             how='left')

assert merged_df.shape[0] == 490, "The number of rows in merged_df is not 482 + 8."
assert merged_df.loc[merged_df['Country'] != 'Taiwan', 
                     ['Urbanization_Rate_2022']].notnull().all().all()
print(f"shape: {merged_df.shape}")

shape: (490, 20)


# Paris Agreement

In [13]:
paris_agreement_iso_codes = [
    "AFG", "ALB", "DZA", "AND", "AGO", "ATG", "ARG", "AUS", "AUT", "AZE", 
    "BHS", "BHR", "BGD", "BRB", "BLR", "BEL", "BLZ", "BEN", "BTN", "BOL", 
    "BIH", "BWA", "BRA", "BRN", "BGR", "BFA", "BDI", "CPV", "KHM", "CMR", 
    "CAN", "CAF", "TCD", "CHN", "COL", "COM", "COG", "CRI", "CIV", "HRV", 
    "CUB", "CYP", "CZE", "PRK", "COD", "DNK", "DJI", "DMA", "DOM", "EGY", 
    "SLV", "GNQ", "ERI", "EST", "ETH", "EUN", "FJI", "FIN", "FRA", "GAB", 
    "GEO", "DEU", "GHA", "GRC", "GRD", "GTM", "GIN", "GNB", "GUY", "HTI", 
    "HND", "HUN", "ISL", "IND", "IDN", "IRN", "IRL", "ISR", "ITA", "JAM", 
    "JPN", "JOR", "KEN", "KIR", "KWT", "LAO", "LVA", "LBN", "LSO", "LBR", 
    "LBY", "LIE", "LTU", "LUX", "MDG", "MYS", "MDV", "MLI", "MLT", "MHL", 
    "MUS", "MRT", "MEX", "FSM", "MCO", "MNG", "MNE", "MAR", "MOZ", "MMR", 
    "NAM", "NRU", "NPL", "NLD", "NZL", "NER", "NOR", "OMN", "PAK", "PLW", 
    "PAN", "PNG", "PRY", "PER", "PHL", "POL", "PRT", "QAT", "KOR", "ROU", 
    "RUS", "RWA", "KNA", "LCA", "VCT", "WSM", "SMR", "STP", "SEN", "SRB", 
    "SGP", "SVK", "SVN", "SLB", "SOM", "ZAF", "SSD", "ESP", "LKA", "PSE", 
    "SDN", "SUR", "SWZ", "SWE", "CHE", "TJK", "THA", "MKD", "TLS", "TON", 
    "TTO", "TUN", "TUR", "TUV", "UGA", "UKR", "ARE", "GBR", "TZA", "USA", 
    "URY", "VUT", "VEN", "VNM", "ZWE"
]

merged_df['Paris_Agreement'] = merged_df['ISO_alpha3'].apply(lambda x: 1 if x in paris_agreement_iso_codes else 0)
assert merged_df.shape[0] == 490, "The number of rows in merged_df is not 482 + 8."


## Region

In [14]:
world_boundaries_url = "https://drive.google.com/uc?id=1k-2ECd2gwJ9FBz1anMRZy7O85uExAFY_"
world_boundaries_path = "../../world-administrative-boundaries.geojson"

gdown.download(world_boundaries_url, world_boundaries_path, quiet=False)
world_boundaries_df = gpd.read_file(world_boundaries_path)
world_boundaries_df = world_boundaries_df[world_boundaries_df['name'] != 'Azores Islands']
world_boundaries_df = world_boundaries_df[["iso3","name", "continent","region"]]
world_boundaries_df.head()

Downloading...
From: https://drive.google.com/uc?id=1k-2ECd2gwJ9FBz1anMRZy7O85uExAFY_
To: /Users/barbaraflores/Desktop/MIDS/IDS798_Capstone/world-administrative-boundaries.geojson
100%|██████████| 8.58M/8.58M [00:00<00:00, 22.2MB/s]


,iso3,name,continent,region
0,MNP,Northern Mariana Islands,Oceania,Micronesia
1,None,Kuril Islands,Asia,Eastern Asia
2,FRA,France,Europe,Western Europe
3,SRB,Serbia,Europe,Southern Europe
4,URY,Uruguay,Americas,South America


In [15]:
merged_df = merged_df.merge(world_boundaries_df, left_on='ISO_alpha3', right_on='iso3', how='left')


merged_df['Region Grouped'] = np.where(
    merged_df['region'] == 'Northern America', 
    'Northern America',  
    np.where(
        merged_df['continent'] == 'Americas', 
        'Central and South America',  
        np.where(
            merged_df['continent'].isin(['Asia', 'Oceania']), 
            'Asia & Oceania',  
            merged_df['continent']  
        )
    )
)
assert merged_df.shape[0] == 490, "The number of rows in merged_df is not 482 +8."
assert merged_df['Region Grouped'].notnull().all(), "The 'Region Grouped' column contains null values."
merged_df.groupby(['continent', 'region' , 'Region Grouped']).size().reset_index(name='count').sort_values(by='Region Grouped')

,continent,region,Region Grouped,count
0,Africa,Eastern Africa,Africa,34
1,Africa,Middle Africa,Africa,18
2,Africa,Northern Africa,Africa,20
3,Africa,Southern Africa,Africa,17
4,Africa,Western Africa,Africa,38
18,Oceania,Australia and New Zealand,Asia & Oceania,11
13,Asia,Western Asia,Asia & Oceania,15
12,Asia,Southern Asia,Asia & Oceania,90
11,Asia,South-Eastern Asia,Asia & Oceania,31
19,Oceania,Melanesia,Asia & Oceania,4


# HDD

In [16]:
HDD_path = "../data/02_interim/HDD_matchedscale.csv"
HDD_df = pd.read_csv(HDD_path)

HDD_df = HDD_df[['geonameid', 'total_year', 'average_year', 'variance_year']]


HDD_df = HDD_df.rename(columns={
    'total_year': 'hdd_total_year',
    'average_year': 'hdd_average_year',
    'variance_year': 'hdd_variance_year'
})
HDD_df = HDD_df.drop_duplicates() # seattle and other cities we have duplicates (original data + new data)
merged_df = merged_df.merge(HDD_df, on='geonameid', how='left')

assert merged_df['hdd_total_year'].notnull().all()
assert merged_df.shape[0] == 490, "The number of rows in merged_df is not 482 +8."
print(f"shape: {merged_df.shape}")


shape: (490, 29)


# 

In [17]:
HDD_path = "../data/02_interim/HDD_matchedscale.csv"
HDD_df = pd.read_csv(HDD_path)

HDD_df = HDD_df[['geonameid', 'total_year', 'average_year', 'variance_year']]

# CDD

In [18]:
CDD_path = "../data/02_interim/CDD_scalematched.csv"
CDD_df = pd.read_csv(CDD_path)

CDD_df = CDD_df[['geonameid', 'total_year', 'average_year', 'variance_year']]

CDD_df = CDD_df.rename(columns={
    'total_year': 'cdd_total_year',
    'average_year': 'cdd_average_year',
    'variance_year': 'cdd_variance_year'
})
CDD_df = CDD_df.drop_duplicates() # seattle and other cities we have duplicates (original data + new data)

merged_df = merged_df.merge(CDD_df, on='geonameid', how='left')
assert merged_df['cdd_total_year'].notnull().all()
assert merged_df.shape[0] == 490, "The number of rows in merged_df is not 482 +8."
print(f"shape: {merged_df.shape}")


shape: (490, 32)


# Weather Data

In [19]:
weather_data_path = "../data/02_interim/temperature_dewpoint_precipitation_2023.csv"
weather_data_df = pd.read_csv(weather_data_path)
weather_data_df = weather_data_df.drop_duplicates() # seattle and other cities we have duplicates (original data + new data)

merged_df = merged_df.merge(weather_data_df, on='geonameid', how='left')
assert merged_df['2m_temperature_2023_avg'].notnull().all() and \
       merged_df['2m_dewpoint_temperature_2023_avg'].notnull().all() and \
       merged_df['total_precipitation_2023'].notnull().all()

assert merged_df.shape[0] == 490, "The number of rows in merged_df is not 482 +8."
print(f"shape: {merged_df.shape}")


shape: (490, 35)


In [20]:
merged_df.columns

Index(['geonameid', 'name_x', 'asciiname', 'latitude', 'longitude',
       'iso alpha 2', 'City', 'Geonames ID', 'Country',
       'Residential EUI (kWh/m2/year)', 'Non-residential EUI (kWh/m2/year)',
       'source', 'ISO_alpha3', 'Population_2023', 'population_density',
       'Subnational HDI', 'Educational index', 'Income index', 'GDP_2022',
       'Urbanization_Rate_2022', 'Paris_Agreement', 'iso3', 'name_y',
       'continent', 'region', 'Region Grouped', 'hdd_total_year',
       'hdd_average_year', 'hdd_variance_year', 'cdd_total_year',
       'cdd_average_year', 'cdd_variance_year', '2m_temperature_2023_avg',
       '2m_dewpoint_temperature_2023_avg', 'total_precipitation_2023'],
      dtype='object')

# Sin Cos Coordinates 

In [21]:

merged_df['longitude_sin'] = np.sin(np.radians(merged_df['longitude']))
merged_df['longitude_cos'] = np.cos(np.radians(merged_df['longitude']))
merged_df['latitude_sin'] = np.sin(np.radians(merged_df['latitude']))
merged_df['latitude_cos'] = np.cos(np.radians(merged_df['latitude']))

assert merged_df['longitude_sin'].notnull().all() and \
       merged_df['longitude_cos'].notnull().all() and \
       merged_df['latitude_sin'].notnull().all() and \
       merged_df['latitude_cos'].notnull().all()

print(f"shape: {merged_df.shape}")
merged_df.head()

shape: (490, 39)


,geonameid,name_x,asciiname,latitude,longitude,iso alpha 2,City,Geonames ID,Country,Residential EUI (kWh/m2/year),...,cdd_total_year,cdd_average_year,cdd_variance_year,2m_temperature_2023_avg,2m_dewpoint_temperature_2023_avg,total_precipitation_2023,longitude_sin,longitude_cos,latitude_sin,latitude_cos
0,292968,Abu Dhabi,Abu Dhabi,24.45118,54.39696,AE,Abu Dhabi,292968,United Arab Emirates,128.447899,...,2188.239273,182.353273,3109.247543,302.02,292.78,0.048,0.813070,0.582166,0.413918,0.910314
1,1138958,Kabul,Kabul,34.52813,69.17233,AF,Kabul,1138958,Afghanistan,213.167026,...,289.296122,24.108010,1951.846629,283.49,272.82,0.378,0.934654,0.355558,0.566811,0.823848
2,3183875,Tirana,Tirana,41.32750,19.81889,AL,Tirana,3183875,Albania,133.717672,...,5.995836,0.499653,2.105448,289.50,283.59,1.327,0.339048,0.940769,0.660362,0.750947
3,616052,Yerevan,Yerevan,40.18111,44.51361,AM,Yerevan,616052,Armenia,198.865302,...,63.188193,5.265683,323.631829,284.26,274.88,0.367,0.701079,0.713084,0.645206,0.764009
4,2240449,Luanda,Luanda,-8.83682,13.23432,AO,Luanda,2240449,Angola,65.343750,...,4224.655207,352.054601,5849.758434,298.57,294.96,0.668,0.228934,0.973442,-0.153621,0.988130


# Image data

In [22]:
image_results_path = "../data/02_interim/image_results_v2.csv"
image_results_df = pd.read_csv(image_results_path)
image_results_df = image_results_df.drop_duplicates()


merged_df = merged_df.merge(image_results_df, left_on=['latitude', 'longitude'], right_on=['lat', 'lon'], how='left')
merged_df = merged_df.drop(columns=['lat', 'lon'])

null_rows = merged_df[merged_df[['pca_result']].isnull().any(axis=1)]

print(f"shape: {merged_df.shape}")
print("")
for index, row in null_rows.iterrows():
    print(f"Row {index} is missing image data.")


shape: (490, 72)

Row 36 is missing image data.
Row 37 is missing image data.
Row 76 is missing image data.
Row 112 is missing image data.
Row 150 is missing image data.
Row 162 is missing image data.
Row 358 is missing image data.
Row 399 is missing image data.
Row 482 is missing image data.
Row 486 is missing image data.
Row 488 is missing image data.


In [23]:
merged_df.head()

,geonameid,name_x,asciiname,latitude,longitude,iso alpha 2,City,Geonames ID,Country,Residential EUI (kWh/m2/year),...,Cluster_20_11,Cluster_20_12,Cluster_20_13,Cluster_20_14,Cluster_20_15,Cluster_20_16,Cluster_20_17,Cluster_20_18,Cluster_20_19,pca_result
0,292968,Abu Dhabi,Abu Dhabi,24.45118,54.39696,AE,Abu Dhabi,292968,United Arab Emirates,128.447899,...,0.050045,0.050037,0.049996,0.050059,0.049936,0.050036,0.050129,0.049987,0.050072,-0.417218
1,1138958,Kabul,Kabul,34.52813,69.17233,AF,Kabul,1138958,Afghanistan,213.167026,...,0.050061,0.050047,0.049996,0.050078,0.049910,0.050046,0.050170,0.049979,0.050096,-0.373100
2,3183875,Tirana,Tirana,41.32750,19.81889,AL,Tirana,3183875,Albania,133.717672,...,0.050065,0.050057,0.049993,0.050093,0.049902,0.050044,0.050187,0.049984,0.050108,-0.258130
3,616052,Yerevan,Yerevan,40.18111,44.51361,AM,Yerevan,616052,Armenia,198.865302,...,0.050213,0.050179,0.049976,0.050292,0.049689,0.050157,0.050623,0.049940,0.050351,-0.899530
4,2240449,Luanda,Luanda,-8.83682,13.23432,AO,Luanda,2240449,Angola,65.343750,...,0.050174,0.050148,0.049981,0.050241,0.049743,0.050124,0.050507,0.049952,0.050287,-0.746722


In [24]:
merged_df.columns

Index(['geonameid', 'name_x', 'asciiname', 'latitude', 'longitude',
       'iso alpha 2', 'City', 'Geonames ID', 'Country',
       'Residential EUI (kWh/m2/year)', 'Non-residential EUI (kWh/m2/year)',
       'source', 'ISO_alpha3', 'Population_2023', 'population_density',
       'Subnational HDI', 'Educational index', 'Income index', 'GDP_2022',
       'Urbanization_Rate_2022', 'Paris_Agreement', 'iso3', 'name_y',
       'continent', 'region', 'Region Grouped', 'hdd_total_year',
       'hdd_average_year', 'hdd_variance_year', 'cdd_total_year',
       'cdd_average_year', 'cdd_variance_year', '2m_temperature_2023_avg',
       '2m_dewpoint_temperature_2023_avg', 'total_precipitation_2023',
       'longitude_sin', 'longitude_cos', 'latitude_sin', 'latitude_cos',
       'Cluster_2_0', 'Cluster_2_1', 'Cluster_10_0', 'Cluster_10_1',
       'Cluster_10_2', 'Cluster_10_3', 'Cluster_10_4', 'Cluster_10_5',
       'Cluster_10_6', 'Cluster_10_7', 'Cluster_10_8', 'Cluster_10_9',
       'Cluster_20_0

# Save File

In [25]:
merged_df.insert(0, 'index', merged_df.index)
output_path = "../data/03_processed"
os.makedirs(output_path, exist_ok=True)
merged_df.to_csv(os.path.join(output_path, "merged_df.csv"), index=False)